# __Title:__ Jupyter notebook for pRESTO
#### __Author:__ Gildas Lepennetier 

<font color='red'> __WARNING__ All those steps are before even starting jupyter </font>

## Set up the servers

This step should be done even before the jupyter server can be started.

### Connect to the server

#### Exemple: LRZ linux cluster + SLURM

Replace by your own credential

`ssh ga94rac2@lxlogin5.lrz.de`


### Load the required modules to have anaconda (conda) available

`module load python`

`conda info -e`

    root                  *  /lrz/sys/intel/studio2019_u3/intelpython2

This will not be enough to actually use pRESTO. One have to create a new environment, using python 3.6
Then, we activate it, and install the required modules

`conda create --name pRESTO python=3.6`

`conda activate pRESTO`

`pip install --upgrade pip presto changeo snakemake jupyter`

Check that everything is properly created:

`conda info -e`

    pRESTO                   /home/hpc/tb601/ga94rac2/.conda/envs/pRESTO
    root                  *  /lrz/sys/intel/studio2019_u3/intelpython2


It is a good practice to regularely run the pip install --upgrade to be up-to-date 

### Start the environment with jupyter + required programs

<font color='red'> WARNING: The modules (conda, jupyter...) have to be activated before you can start the server</font>

To be able to use jupyter notebook, the proper modules have to be started

`conda activate pRESTO`

`conda info -e`

    pRESTO                *  /home/hpc/tb601/ga94rac2/.conda/envs/pRESTO
    root                     /lrz/sys/intel/studio2019_u3/intelpython2

To quit the session after, the command will be `source deactivate pRESTO`


## Use Git versioning

In the following directory

`cd ~/notebooks_jupyter`

Start a git, if not already present. First, go to github and create the directory. Then, execute:

`module load git
git init
echo ".ipynb_checkpoints/" > .gitignore

git remote add origin git@github.com:GildasLepennetier/notebooks_jupyter
git remote add origin https://github.com/GildasLepennetier/notebooks_jupyter.git

git add * .gitignore
git commit -am "first commit"

git push -u origin master
`



## Start the jupyter server


<font color='blue'> optional: change directory, go where the notebooks are stored</font>

`cd ~/notebooks_jupyter`

`jupyter notebook --port-retries=10 --no-browser --port=8888 small_tests.ipynb`

<font color='orange'> WARNING: If the port is already used, the program will try another one, in this case the following step have to take the new port into account</font>

Start a ssh tunneling (using the port announced) <font color='red'>on the local computer (not the server) </font> to be able to use your own browser

`ssh -v -N -L localhost:8888:localhost:8888 ga94rac2@lxlogin5.lrz.de`

## Check some variables:

In [1]:
%%bash
echo -e "local working directory:\t$(pwd)"
echo -e "host is:\t\t\t$(hostname -i)"


local working directory:	/home/hpc/tb601/ga94rac2/notebooks_jupyter
host is:			10.156.79.105


## Update the environment

It is a good idea to update systematically the programs used

In [6]:
!pip install --upgrade presto changeo | cut -c 1-100

Requirement already up-to-date: presto in /home/hpc/tb601/ga94rac2/.conda/envs/pRESTO/lib/python3.6/
Requirement already up-to-date: changeo in /home/hpc/tb601/ga94rac2/.conda/envs/pRESTO/lib/python3.6


## PARAMETERS

It is a good practice to have files with all parameters for a job. A common was to do it is to have a file in: 

    text
    json
    python pickle
    R data

All options are retrived from those files and help to keep track of the data for each run
